In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import plotly.graph_objects as go

from scipy.stats import chi2_contingency
import missingno as msno
pd.set_option('display.max_columns', None)

df = pd.read_csv('DataFrame_Crédito.csv').drop(columns=['Unnamed: 0', 'id'])
df.head()

,idade,sexo,dependentes,escolaridade,estado_civil,grupo_etario,idade_faixa,idade_estado_civil,escolaridade_renda,salario_anual,limite_credito,renda_alta,meses_de_relacionamento,interacoes_12m,meses_inativo_12m,valor_transacoes_12m,qtd_transacoes_12m,transacoes_por_mes,meses_ativos_12m,tipo_cartao,taxa_utilizacao_credito,valor_medio_transacao,frequencia_interacao,qtd_produtos,default
0,45,M,3,ensino medio,casado,Adulto,38-45,45 - casado,ensino medio - $60K - $80K,$60K - $80K,12691.51,0,39,3,1,1144.90,42,3.500000,11,blue,0.090210,27.259524,0.272727,5,0
1,49,F,5,mestrado,solteiro,Adulto,46-51,49 - solteiro,mestrado - menos que $40K,menos que $40K,8256.96,0,44,2,1,1291.45,33,2.750000,11,blue,0.156407,39.134848,0.181818,6,0
2,51,M,3,mestrado,casado,Adulto,46-51,51 - casado,mestrado - $80K - $120K,$80K - $120K,3418.56,1,36,0,1,1887.72,20,1.666667,11,blue,0.552197,94.386000,0.000000,4,0
3,40,M,3,sem educacao formal,casado,Adulto,38-45,40 - casado,sem educacao formal - $60K - $80K,$60K - $80K,4716.22,0,21,0,1,816.08,28,2.333333,11,blue,0.173037,29.145714,0.000000,5,0
4,44,M,2,mestrado,casado,Adulto,38-45,44 - casado,mestrado - $40K - $60K,$40K - $60K,4010.69,0,36,2,1,1088.07,24,2.000000,11,blue,0.271292,45.336250,0.181818,3,0


In [2]:
def plot_scatter_plots(df, pairs, hue_col, num_cols=3):
    """
    Plota gráficos de dispersão para pares de variáveis numéricas especificadas de um DataFrame.

    Parâmetros:
    df (pd.DataFrame): O DataFrame contendo os dados.
    pairs (list of tuples): Lista de tuplas onde cada tupla contém dois nomes de colunas a serem comparadas.
    num_cols (int): O número de colunas no layout dos subplots.
    """
    num_rows = -(-len(pairs) // num_cols)  # arredonda para cima

    # Criando a figura e os subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 5*num_rows))
    axes = axes.flatten()  # Achata a matriz de eixos para facilitar a iteração

    if hue_col == 'null':
        # Função para plotar o gráfico de dispersão em um subplot específico
        def scatterplot(ax, x_col, y_col):
            sns.scatterplot(x=df[x_col], y=df[y_col], ax=ax)
            ax.set_title(f'Dispersão {x_col} vs {y_col}')
            ax.set_xlabel(x_col)
            ax.set_ylabel(y_col)
    else:
        # Função para plotar o gráfico de dispersão em um subplot específico
        def scatterplot(ax, x_col, y_col):
            sns.scatterplot(x=df[x_col], y=df[y_col], hue=df[hue_col], ax=ax)
            ax.set_title(f'Dispersão {x_col} vs {y_col}')
            ax.set_xlabel(x_col)
            ax.set_ylabel(y_col)

    # Iterando sobre os pares de colunas e os eixos
    for i, (x_col, y_col) in enumerate(pairs):
        scatterplot(axes[i], x_col, y_col)

    # Removendo subplots vazios
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    # Ajustando o layout para evitar sobreposição
    plt.tight_layout()
    plt.show()

#import itertools

# Lista de colunas
columns = df.select_dtypes('number').columns

def criar_pares(lista):
  pares = []
  for i in range(len(lista)):
    for j in range(i + 1, len(lista)):
      pares.append((lista[i], lista[j]))
  return pares

In [3]:
def AB2N(dataframe, x_coluna, y_coluna):
    tabela = df.groupby(x_coluna)[y_coluna].mean().to_dict()
    fig = px.line(x=list(tabela.keys()), y=list(tabela.values()), title=f'Média de: {x_coluna} vs {y_coluna}', 
                labels={'x': x_coluna, 'y': 'Média de ' + y_coluna})
    fig.update_layout(template='plotly_white')
    
    fig.show()

In [4]:
df.select_dtypes('number').columns

Index(['idade', 'dependentes', 'limite_credito', 'renda_alta',
       'meses_de_relacionamento', 'interacoes_12m', 'meses_inativo_12m',
       'valor_transacoes_12m', 'qtd_transacoes_12m', 'transacoes_por_mes',
       'meses_ativos_12m', 'taxa_utilizacao_credito', 'valor_medio_transacao',
       'frequencia_interacao', 'qtd_produtos', 'default'],
      dtype='object')

## Como são as transações

In [5]:
AB2N(df, 'qtd_transacoes_12m', 'valor_medio_transacao')
AB2N(df, 'interacoes_12m', 'limite_credito')